In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [1]:
from transformers import BertTokenizerFast, BertForPreTraining,TFBertForTokenClassification ,TFTrainer, TFTrainingArguments
import os
import tensorflow as tf
import pandas as pd
import datasets

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from datasets import load_dataset
dataset = load_dataset("german_legal_entity_recognition","all",ignore_verifications =True , download_mode="force_redownload")

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.39M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset german_legal_entity_recognition downloaded and prepared to /root/.cache/huggingface/datasets/german_legal_entity_recognition/all/1.0.0/1d4ec33053d6b0f788de20aa9010bf6f5a16ea6682b83ddd79ab82773348658c. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset = dataset["train"].train_test_split(test_size=0.2)

In [5]:
labels = dataset["train"].features[f"ner_tags"].feature.names

In [6]:
len(labels) 

39

In [7]:
#Load Model and Tokenizer
from transformers import AutoTokenizer
with tf.device('GPU'):
    tokenizer = BertTokenizerFast.from_pretrained('deepset/gbert-large')
    model = TFBertForTokenClassification.from_pretrained('deepset/gbert-large', num_labels=len(labels) )

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at deepset/gbert-large and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_and_align_labels(examples):
    #print(examples.keys())
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [10]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/54 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [11]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [12]:
from datasets import Dataset

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer,max_length=512 ,padding='max_length', return_tensors="tf",label_pad_token_id =-100)

tf_train_set  = full_train_dataset.to_tf_dataset(
    columns=["attention_mask","token_type_ids", "input_ids", "labels"],
    shuffle=True,
    batch_size=6,
    collate_fn=data_collator,
)

tf_test_set  = full_eval_dataset.to_tf_dataset(
    columns=["attention_mask","token_type_ids", "input_ids", "labels"],
    shuffle=True,
    batch_size=6,
    collate_fn=data_collator,
)


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [ ]:
from transformers import create_optimizer

batch_size = 6
num_train_epochs = 3
num_train_steps = (len(full_train_dataset) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [ ]:
import tensorflow as tf
model.compile(optimizer=optimizer  )

In [ ]:
history = model.fit(
    tf_train_set,
    
    epochs=num_train_epochs
)

In [ ]:
model.evaluate(tf_test_set)

In [ ]:
model.save_pretrained('./Legal_NER/')

In [ ]:
tokenizer.save_pretrained('./Legal_TOK/')